<a href="https://colab.research.google.com/github/jmlDC/MediaBias-Thesis22-23/blob/Developing-Tool/Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MEDIA BIAS Thesis

# Setting up Environment

## PIP install

In [1]:
!pip3 install requests-html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 4.8 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1270 sha256=052b6b382468fca40ab0476bb65072ba8d14abf63a3b201df8b02db579045d70
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24589 sha256=c1203e848da816061929f8fdb3a4fb4330275a2e76e249a548fee341d2031312
  Stored in directory: /root/.cache/pip/wheels/70/4b/f0/eaf5a8de646d8676dc25caa01949b9f9d883b8fa2efb435bc3
Successfully built bs4 parse


## Imports

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium

# Install xvfb
apt install -y xvfb

# Install Selenium-Profiles
pip uninstall -y selenium_profiles
pip install --no-cache-dir selenium_profiles>=2.2.6

# pip install https://github.com/kaliiiiiiiiii/Selenium-Profiles/archive/refs/heads/dev.zip # dev-branch

# install python packages
pip install google-colab-shell
pip install webdriver-manager
pip install Pyvirtualdisplay 

Executing: /tmp/apt-key-gpghome.CtmgeB1PLG/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.LWJhJ1Hm4w/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.RpcwTKjYyi/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/l

In [4]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import date
from requests_html import HTMLSession 
import re

/usr/local/lib/python3.10/dist-packages/trio/_core/_multierror.py:406: RuntimeWarning: IPython detected, but you already have a custom exception handler installed. I'll skip installing Trio's custom handler, but this means exception groups will not show full tracebacks.
  warnings.warn(


## Mounting to GDrive

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### dir variable

In [6]:
dir  = "/content/gdrive/MyDrive/THESIS-MS/Git-Thesis22-23/Official/"

# Preperation Scrape

1. GMA7 - infinite scroll with dates
2. ABS CBN (Online) - has mix of Filipino news **
3. CNN Philippines - limited newslink  **
4. Philippine Star -  headline/nation infinite scroll
5. Rappler - by page
6. Manila Bulletin - infinite more button
7. TV5 - single page (?) limited **
8. Manila Standard - limited load more *
9. Sunstar Philippines - pages 
10. Philippine News Agency

## Extract Source Function

In [7]:
def extract_source(url):
    agent = {"User-Agent":"Chrome/105.0.0.0"}
    try:
      source=requests.get(url, headers=agent)
    except Exception as e:                                   # this describes what to do if an exception is thrown
      error_type, error_obj, error_info = sys.exc_info()      # get the exception information
      print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
      print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
    
    return source                                               #ignore this page. Abandon this and go back.

## Chrome driver

In [9]:
# https://colab.research.google.com/github/kaliiiiiiiiii/Selenium-Profiles/blob/master/google-colab/selenium_profiles.ipynb#scrollTo=lThF-0LvpZf3
## @title Start actual driver
from selenium_profiles.webdriver import Chrome
from selenium_profiles.profiles import profiles
from selenium.webdriver.common.by import By  # locate elements
from selenium_profiles.utils.colab_utils import display, showscreen, show_html # virtual display
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotVisibleException, ElementNotSelectableException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

display = display()

def startChromeDriver():
  options = Options()
#   options.add_argument('--disable-blink-features=AutomationControlled')
  chromedriver_path = ChromeDriverManager(version="90.0.4430.24").install()
  profile = profiles.Windows() # or .Android
  profile["cdp"]["cores"] = None # Chrome 90 doesn't allow emulating cores :(driver = mydriver.start(profile, uc_driver=False, executable_path=chromedriver_path)
  mydriver = Chrome(profile, executable_path=chromedriver_path, options=options)
  display.start_display()
  return mydriver.start()
   

# Media Outlet

## ABS CBN *

In [ ]:
## ABS CBN
# pagesToGet= 2
# topic = "Queen Elizabeth Death"
site = "ABS-CBN_NEWS"


data=pd.DataFrame(columns=['Title','Link','Date'])

# for page in range(1,pagesToGet+1):
#   url = 'https://news.abs-cbn.com/special-pages/search?q='+topic.replace(" ", "%20")+'&gsc.sort=&gsc.page='+str(page)+'#gsc.tab=0&gsc.q='+topic.replace(" ", "%20")+'&gsc.sort=&gsc.page='+str(page)
#   print(url)

#   driver.get(url)
#   # WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.ID, '___gcse_0'))) 
#   source = driver.page_source

#   time.sleep(2)   
#   soup = BeautifulSoup(source, 'html.parser')
#   searches = soup.find_all('div', {'class':'gsc-webResult gsc-result'})
#   filename= f"{dir}{site}-{topic}-NEWS-List.csv"     

#   for x in searches:
#     frame=[]
#     Title = x.find('div', class_="gs-title").find("a").text.strip()
#     Date = x.find('div', class_="gs-bidi-start-align gs-snippet").text[:12].strip()
#     Link = x.find('a', class_="gs-title")['href']
#     frame.extend((Title, Link, Date, topic))

#     data = data.append(pd.Series(frame, index=data.columns[:len(frame)]), ignore_index=True)



# data.to_csv(filename)
# driver.quit()
# data




In [ ]:
def abscbnArticleScraper(url):
  print(url)
  driver = startChromeDriver()
  driver.get(url)
  WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'news-title'))) 

  source = driver.page_source
  soup=BeautifulSoup(source, 'html.parser')
  row=[]
 
  Title = soup.find('h1',{'class':'news-title'}).text.strip()
  Author = soup.find('span',attrs={'class':'editor'}).text.strip()
  Date = soup.find('span', attrs={'class': 'date-posted'}).text[:-8].strip() 
  
  textList =soup.find("div", {'class' :'article-content'}).find_all("p")
  body = ""
  for t in textList:
    if (t.text != "RELATED VIDEO:"):
      body += (t.text) +"\n" 

  row.extend((Title, Author, Date, body))
  return row

site='ABS-CBN'
data2 = pd.DataFrame(columns=["Title", "Author", "Date", "Text"])
filename= f"{dir+site}_{date.today()}NEWS.csv"

row = abscbnArticleScraper('https://news.abs-cbn.com/news/05/05/23/ph-airspace-shutdown-on-may-17-shortened-to-2-hours')
data2 = data2.append(pd.Series(row, index=data2.columns[:len(row)]), ignore_index=True)

  
data2.to_csv(filename)
data2.head()

https://news.abs-cbn.com/news/05/05/23/ph-airspace-shutdown-on-may-17-shortened-to-2-hours


,Title,Author,Date,Text
0,Philippine airspace shutdown on May 17 shorten...,ABS-CBN News,May 05 2023,Philippine airspace shutdown on May 17 shorten...


## CNN Philippines *

In [ ]:
site = "CNN-Philippines"
data=pd.DataFrame(columns=['Statement','Link','Date'])
url = 'https://www.cnnphilippines.com/news/'
print(url)

# driver = startChromeDriver()
# driver.get(url)
# driver.implicitly_wait(10)

# source = driver.page_source
# soup = BeautifulSoup(source, 'html.parser')

# linksCarousel = soup.find("ul", {'id' : 'grid_thumbnail_stories'}).find_all('li', {'class':'story left-grid'})
# linksRow = soup.find("ul", {'id' : 'grid_thumbnail_stories'}).find_all('li', {'class':'story right-grid'})
# links = linksCarousel + linksRow
# print(len(links))

# filename= f"{dir}{site}_{date.today()}_NEWS_LinkList.csv"    

# for j in links:
#     Statement = j.find('div').find('div', {'class': 'story'}).find('div', {'class': 'story_title_holder'}).find('div', {'class' : 'story_title'}).text[:-21].strip()
#     Link = j.find('a')['href'].strip()
#     Date = j.find('div', {'class' : 'story_title_holder'}).find('div', {'class' : 'story_title'}).find('div', {'class': 'archive_date_time'}).text[:-8].strip()
#     df_new_row = { 'Statement': Statement, 'Link': Link, 'Date': Date}
#     print(Statement, Link, Date)
#     data = data.append(pd.Series(df_new_row, index=data.columns[:len(df_new_row)]), ignore_index=True)

# driver.quit()
# data.to_csv(filename)
# data.head()

# owl-pagination

In [ ]:
def cnnArticleScraper(url):
  print(url)
  driver = startChromeDriver()
  driver.get(url)
  WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'title'))) 
  source = driver.page_source
  soup=BeautifulSoup(source, 'html.parser')
  row=[]
  try: 
    Title = soup.find('h1',{'class':'title'}).text.strip()
    Author = soup.find('div',attrs={'class':'author-byline'}).find('p').find('a').text.strip()
    Date = soup.find('div', attrs={'class': 'dateLine'}).find('p', {'class': 'dateString no-icon'}).text[10:-12].strip() 
    textList =soup.find("div", {'class' :'article-maincontent-p cnn-life-body'}).find_all("p")
    body = ""
    for t in textList:
        if (t.text != "ADVERTISEMENT"):
          body += (t.text) +"\n" 
    row.extend((Title, Author, Date, body))
  except AttributeError:
    pass
  driver.quit()
  return row

site = "CNN-Philippines"
data2 = pd.DataFrame(columns=["Title", "Author", "Date", "Text"])
filename= f"{dir+site}_{date.today()}NEWS.csv"

row = cnnArticleScraper('https://www.cnnphilippines.com/news/2023/5/4/MIAA-possible-extension-of-May-17-airspace-shutdown.html')
data2 = data2.append(pd.Series(row, index=data2.columns[:len(row)]), ignore_index=True)
  
data2.to_csv(filename)
data2.head()

https://www.cnnphilippines.com/news/2023/5/4/MIAA-possible-extension-of-May-17-airspace-shutdown.html


,Title,Author,Date,Text
0,MIAA prepares for possible extension of 6-hour...,CNN Philippines Staff,"May 4, 2023,","Metro Manila (CNN Philippines, May 4) — The Ma..."


In [ ]:
print(BeautifulSoup(extract_source('https://www.cnnphilippines.com/news/2023/5/4/us-backs-ph-south-china-sea.html').text, 'html.parser').prettify())

## Philippine Star *

In [ ]:
# https://www.philstar.com/headlines
# https://www.philstar.com/nation˛

site = "PhilStar"
data=pd.DataFrame(columns=['Statement','Link','Date'])
url = 'https://www.philstar.com/headlines/'
print(url)

driver = startChromeDriver()
driver.get(url)
driver.implicitly_wait(10)
counter = 2
try:
  SCROLL_PAUSE_TIME = 1
  cnt = counter
  # Get scroll height
  last_height = driver.execute_script("return document.body.scrollHeight")
  while cnt>0:
      # Scroll down to bottom
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      # Wait to load page
      time.sleep(SCROLL_PAUSE_TIME)
      # Calculate new scroll height and compare with last scroll height
      new_height = driver.execute_script("return document.body.scrollHeight")
      if new_height == last_height:
          break
      last_height = new_height
      print("scrolling...")
    #   showscreen(driver)
      cnt -= 1
except Exception as e:                                   
  error_type, error_obj, error_info = sys.exc_info()      
  print ('ERROR FOR LINK:',url)                          
  print (error_type, 'Line:', error_info.tb_lineno)  

try:
  WebDriverWait(driver, timeout=10).until(EC.presence_of_element_located((By.CLASS_NAME, 'theContent')))
  WebDriverWait(driver, timeout=10).until(EC.presence_of_element_located((By.ID, 'news_main')))
  WebDriverWait(driver, timeout=10).until(EC.presence_of_element_located((By.CLASS_NAME, 'jscroll-added')))
  WebDriverWait(driver, timeout=10).until(EC.presence_of_element_located((By.CLASS_NAME, 'jscroll-inner')))
  WebDriverWait(driver, timeout=10).until(EC.presence_of_element_located((By.CLASS_NAME, 'news_column trending')))

  source = driver.page_source
  soup = BeautifulSoup(source, 'html.parser')
  linksCarousel  = soup.find('div', {'class':'carousel__items'}).find_all('div', {'class' : 'carousel__item__title'})
  links1 = []
  links1 += soup.find('div', {'id':'news_main'}).find('div', {'class' : 'news_column trending'}).find_all('div', {'class':'tiles trend'}) 
  links1 += soup.find('div', {'id':'news_main'}).find('div', {'class' : 'news_column latest'}).find_all('div', {'class':'tiles late'}) 
  
  links2 = []
  if (counter >1):
    linksAddJScroll = soup.find('div', {'id':'news_main'}).find_all('div', {'class':'jscroll-added'})
    # showscreen(driver)
    for x in linksAddJScroll:
        try:
            # js_code = "arguments[0].scrollIntoView();"
            # # //*[@id="news_main"]/div/div[6]/div[2]
            # element = WebDriverWait(driver, timeout=5).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="news_main"]/div/div[6]/div[2]')))
            time.sleep(2)
            links2 += x.find('div', {'class' : 'news_column trending'}).find_all('div', attrs={'class': re.compile('^tiles trend')})
            links2 += x.find('div', {'class' : 'news_column latest'}).find_all('div', attrs={'class': re.compile('^tiles late')})
        except:
            print('help!!!!!!!')
            error_type, error_obj, error_info = sys.exc_info()      
            print ('ERROR FOR LINK:',url)                          
            print (error_type, 'Line:', error_info.tb_lineno) 

#   showscreen(driver)
#   print(linksTrending,linksLatest)
except Exception as e:                                   
  error_type, error_obj, error_info = sys.exc_info()      
  print ('ERROR FOR LINK:',url)                          
  print (error_type, 'Line:', error_info.tb_lineno) 

print(len(linksCarousel)+len(links1)+len(links2))

for j in linksCarousel:
    Statement = j.find('h2').find('a').text.strip()
    Link = j.find('a')['href'].strip()
    Date = soup.find('div', {'class':'carousel__items'}).find('div', {'class' : 'carousel__item__time'}).text
    df_new_row = { 'Statement': Statement, 'Link': Link, 'Date': Date}
    print('car',Statement, Link, Date)
    data = data.append(pd.Series(df_new_row, index=data.columns[:len(df_new_row)]), ignore_index=True)

for j in links1:
    Statement = j.find('div', {'class': 'TilesText spec'}).find('div', {'class':'news_title'}).find('h2').find('a').text.strip()
    Link = j.find('div', {'class': 'TilesText spec'}).find('div', {'class':'news_title'}).find('a')['href'].strip()
    Date = j.find('div', {'class': 'TilesText spec'}).find('div', {'class':'dateOfFeature'}).text
    df_new_row = { 'Statement': Statement, 'Link': Link, 'Date': Date}
    print('li',Statement, Link, Date)
    data = data.append(pd.Series(df_new_row, index=data.columns[:len(df_new_row)]), ignore_index=True)

for j in links2:
    Statement = j.find('div', {'class': 'TilesText spec'}).find('div', {'class':'news_title'}).find('a').text.strip()
    Link = j.find('div', {'class': 'TilesText spec'}).find('div', {'class':'news_title'}).find('a')['href'].strip()
    Date = j.find('div', {'class': 'TilesText spec'}).find('div', {'class':'dateOfFeature'}).text
    df_new_row = { 'Statement': Statement, 'Link': Link, 'Date': Date}
    print('li2',Statement, Link, Date)
    data = data.append(pd.Series(df_new_row, index=data.columns[:len(df_new_row)]), ignore_index=True)

display.stop_display()
driver.quit()

filename= f"{dir}{site}_{date.today()}_NEWS_LinkList.csv"   
data.to_csv(filename)
data

In [ ]:
def philStarArticleScraper(url):
  print(url)
  row=[]

  try: 
    driver = startChromeDriver()
    driver.get(url)
    WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, 'article__title'))) 
    source = driver.page_source
    soup=BeautifulSoup(source, 'html.parser')

    WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.ID, 'sports_article_writeup'))) 
    Title = soup.find('div',{'class':'article__title'}).find('h1').text.strip()
    Author = soup.find('div', attrs={'class': 'article__credits'}).find('div',attrs={'class':'article__credits-author-pub'}).text.strip()
    Date = soup.find('div', attrs={'class': 'article__credits'}).find('div', {'class': 'article__date-published'}).text[:-11].strip() 
    textList =soup.find("div", {'class' :'article__writeup'}).find_all("p")
    body = ""
    for t in textList:
        body += (t.text) +"\n" 
    row.extend((Title, Author, Date, body))
    
  except Exception as e:                                   
      error_type, error_obj, error_info = sys.exc_info()      
      print ('ERROR FOR LINK:',url)                          
      print (error_type, 'Line:', error_info.tb_lineno)
  driver.quit()
  return row

site = "PhilStar"
data2 = pd.DataFrame(columns=["Title", "Author", "Date", "Text"])
filename= f"{dir+site}_{date.today()}NEWS.csv"

for index, row in data.iterrows():
  url = row["Link"] 
  row = philStarArticleScraper(url)
  data2 = data2.append(pd.Series(row, index=data2.columns[:len(row)]), ignore_index=True)
  
data2.to_csv(filename)
data2

https://www.philstar.com/headlines/2023/05/05/2263946/comelec-seeks-budget-hybrid-count-machines
https://www.philstar.com/headlines/2023/05/05/2263943/philippines-us-increase-tempo-sea-activities
https://www.philstar.com/headlines/2023/05/05/2263939/airlines-ready-may-17-airspace-closure
https://www.philstar.com/headlines/2023/05/04/2263761/two-deped-admin-officials-resign-ablan-says-quitting-not-related-laptops-controversy
https://www.philstar.com/headlines/2023/05/05/2263935/42-want-rotc-be-optional-sws
https://www.philstar.com/headlines/2023/05/04/2263627/baguio-cebu-suspend-face-face-classes-due-covid-19
https://www.philstar.com/headlines/2023/05/03/2263348/2-police-generals-2-colonels-recommended-drug-raps
https://www.philstar.com/headlines/2023/05/02/2263253/over-150-filipinos-evacuated-sudan-arrive-may-4
https://www.philstar.com/headlines/2023/05/04/2263763/dilg-identifies-alternative-fishing-sites-oriental-mindoro-fishers
https://www.philstar.com/headlines/2023/05/04/2263655/go

,Title,Author,Date,Text
0,Comelec seeks budget for hybrid count machines,Evelyn Macairan - The Philippine Star This con...,"May 5, 202","MANILA, Philippines — To prevent a repeat of m..."
1,"Philippines, US to increase tempo of sea activ...",Alexis Romero - The Philippine Star This conte...,"May 5, 202",WASHINGTON – The Philippines has agreed to an ...
2,Airlines ready for May 17 airspace closure,Elijah Felice Rosales - The Philippine Star Th...,"May 5, 202","MANILA, Philippines — Airlines have started re..."
3,Two DepEd admin officials resign; Ablan says q...,Cristina Chi - Philstar.com,"May 4, 20","MANILA, Philippines — Two Department of Educat..."
4,42% want ROTC to be optional – SWS,Janvic Mateo - The Philippine Star This conten...,"May 5, 202","MANILA, Philippines — More Filipinos prefer an..."
5,"UP Baguio, Cebu suspend face-to-face classes d...",Janvic Mateo - The Philippine Star This conten...,"May 4, 202","MANILA, Philippines — The University of the Ph..."
6,"2 police generals, 2 colonels recommended for ...",Emmanuel Tupas - The Philippine Star This cont...,"May 3, 202","MANILA, Philippines — Four ranking police offi..."
7,Over 150 Filipinos evacuated from Sudan to arr...,Philstar.com,"May 2, 20","MANILA, Philippines — More Filipinos who were ..."
8,DILG identifies alternative fishing sites for ...,Gaea Katreena Cabico - Philstar.com,"May 4, 20","MANILA, Philippines — The Department of the In..."
9,Government printer to produce driver’s license...,Bella Cariaso - The Philippine Star This conte...,"May 4, 202","MANILA, Philippines — To speed up the release ..."


## TV5


In [ ]:
# https://news.tv5.com.ph/articles/politics

site = "TV5"
data=pd.DataFrame(columns=['Statement','Link','Date'])
url = 'https://news.tv5.com.ph/articles/politics'
print(url)

driver = startChromeDriver()
driver.get(url)
driver.implicitly_wait(10)

WebDriverWait(driver, timeout=30).until(EC.visibility_of_element_located((By.CLASS_NAME, 'col-md-8')))
WebDriverWait(driver, timeout=5).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="grid-cat"]/div[2]')))

source = driver.page_source
time.sleep(2)
soup = BeautifulSoup(source, 'html.parser')

links = soup.find('div',{'class':'col-md-8'}).find_all('a')
print(len(links))

for j in links:
    Statement = j.find('div', {'class' : 'title'}).text.strip()
    Link = j['href'].strip()
    Date = j.find('div', {'class': 'date'}).text.strip()
    df_new_row = { 'Statement': Statement, 'Link': Link, 'Date': Date}
    data = data.append(pd.Series(df_new_row, index=data.columns[:len(df_new_row)]), ignore_index=True)

driver.quit()
data.head()

https://news.tv5.com.ph/articles/politics
50


,Statement,Link,Date
0,RUTHLESS APPROACH | Marcos says Duterte’s drug...,https://news.tv5.com.ph/politics/read/ruthless...,"May 5, 2023 , 10:34 AM"
1,MARITIME DISPUTE | U.S. issues guidelines on d...,https://news.tv5.com.ph/politics/read/maritime...,"May 4, 2023 , 07:30 PM"
2,‘BROADER UNDERSTANDING’ | Same-sex union part ...,https://news.tv5.com.ph/politics/read/broader-...,"May 4, 2023 , 10:29 AM"
3,SERIOUS ALLEGATION | Russia says Ukraine tried...,https://news.tv5.com.ph/politics/read/serious-...,"May 4, 2023 , 09:38 AM"
4,WEST PHILIPPINE SEA | US stands with Philippin...,https://news.tv5.com.ph/politics/read/west-phi...,"May 3, 2023 , 09:49 PM"


In [ ]:
# https://news.tv5.com.ph/articles/national

site = "TV5"
url = 'https://news.tv5.com.ph/articles/national'
print(url)

driver = startChromeDriver()
driver.get(url)
driver.implicitly_wait(10)

WebDriverWait(driver, timeout=30).until(EC.visibility_of_element_located((By.CLASS_NAME, 'col-md-8')))
WebDriverWait(driver, timeout=5).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="grid-cat"]/div[2]')))

source = driver.page_source
time.sleep(2)
soup = BeautifulSoup(source, 'html.parser')

links = soup.find('div',{'class':'col-md-8'}).find_all('a')
print(len(links))

for j in links:
    Statement = j.find('div', {'class' : 'title'}).text.strip()
    Link = j['href'].strip()
    Date = j.find('div', {'class': 'date'}).text.strip()
    df_new_row = { 'Statement': Statement, 'Link': Link, 'Date': Date}
    data = data.append(pd.Series(df_new_row, index=data.columns[:len(df_new_row)]), ignore_index=True)

driver.quit()
filename= f"{dir}{site}_{date.today()}_NEWS_LinkList.csv"    
data.to_csv(filename)
data.head()

https://news.tv5.com.ph/articles/national
50


,Statement,Link,Date
0,RUTHLESS APPROACH | Marcos says Duterte’s drug...,https://news.tv5.com.ph/politics/read/ruthless...,"May 5, 2023 , 10:34 AM"
1,MARITIME DISPUTE | U.S. issues guidelines on d...,https://news.tv5.com.ph/politics/read/maritime...,"May 4, 2023 , 07:30 PM"
2,‘BROADER UNDERSTANDING’ | Same-sex union part ...,https://news.tv5.com.ph/politics/read/broader-...,"May 4, 2023 , 10:29 AM"
3,SERIOUS ALLEGATION | Russia says Ukraine tried...,https://news.tv5.com.ph/politics/read/serious-...,"May 4, 2023 , 09:38 AM"
4,WEST PHILIPPINE SEA | US stands with Philippin...,https://news.tv5.com.ph/politics/read/west-phi...,"May 3, 2023 , 09:49 PM"


In [ ]:
# preprocess TV text in future
def tv5ArticleScraper(url):
  print(url)
  row=[]

  try: 
    driver = startChromeDriver()
    driver.get(url)
    WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, 'post-title'))) 
    source = driver.page_source
    soup=BeautifulSoup(source, 'html.parser')

    Title = soup.find('div',{'class':'post-title'}).text.strip()
    try:
        Author = soup.find('em').text.strip()
    except AttributeError:
        Author = ""
    try:
        Date = soup.find('div' , {'class', 'post-postdata'}).text[:-10].strip() 
    except Exception as e:                                   
      error_type, error_obj, error_info = sys.exc_info()      
      print ('ERROR FOR LINK:',url)                          
      print (error_type, 'Line:', error_info.tb_lineno)
      Date = ""
      
    textList =soup.find("div", {'class' :'post-cont'}).find_all("p")
    body = ""
    for t in textList:
        body += (t.text) +"\n" 
    row.extend((Title, Author, Date, body))
    
  except Exception as e:                                   
      error_type, error_obj, error_info = sys.exc_info()      
      print ('ERROR FOR LINK:',url)                          
      print (error_type, 'Line:', error_info.tb_lineno)
  driver.quit()
  return row

site = "TV5"
data2 = pd.DataFrame(columns=["Title", "Author", "Date", "Text"])
filename= f"{dir+site}_{date.today()}NEWS.csv"

for index, r in data.iterrows():
  url = r["Link"] 
  row = tv5ArticleScraper(url)
  data2 = data2.append(pd.Series(row, index=data2.columns[:len(row)]), ignore_index=True)
  if index ==10:
      break
 
data2.to_csv(filename)
data2

https://news.tv5.com.ph/politics/read/ruthless-approach-marcos-says-dutertes-drug-war-had-abuses
https://news.tv5.com.ph/politics/read/maritime-dispute-u-s-issues-guidelines-on-defending-philippines-from-south-china-sea-attack
https://news.tv5.com.ph/politics/read/broader-understanding-same-sex-union-part-of-curriculum-since-2013-says-deped
https://news.tv5.com.ph/politics/read/serious-allegation-russia-says-ukraine-tried-to-kill-putin-with-night-time-drone-attack
https://news.tv5.com.ph/politics/read/west-philippine-sea-us-stands-with-philippines-against-chinese-intimidation-in-s-china-sea-official
https://news.tv5.com.ph/politics/read/true-friendship-marcos-cites-need-to-continually-revisit-trade-ties-between-philippines-us
https://news.tv5.com.ph/politics/read/us-philippine-ties-amid-china-pressure-us-and-philippines-recommit-to-security-alliance
https://news.tv5.com.ph/politics/read/direct-line-philippines-china-to-discuss-fishing-rights-in-south-china-sea-says-marcos
https://news.

,Title,Author,Date,Text
0,RUTHLESS APPROACH | Marcos says Duterte’s drug...,By Beatrice Puente,"May 5, 2023 ,","By Beatrice Puente\n(May 5, 2023) – President ..."
1,MARITIME DISPUTE | U.S. issues guidelines on d...,By Karen Lema,"May 4, 2023 ,","By Karen Lema\nMANILA, May 4 (Reuters) – The U..."
2,‘BROADER UNDERSTANDING’ | Same-sex union part ...,By Beatrice Puente,"May 4, 2023 ,","By Beatrice Puente\n(May 4, 2023) – The Depart..."
3,SERIOUS ALLEGATION | Russia says Ukraine tried...,(Additional reporting by Felix Light and Jake ...,"May 4, 2023 ,",By Mark Trevelyan\nMay 3 (Reuters) – Russia ac...
4,WEST PHILIPPINE SEA | US stands with Philippin...,By Karen Lema,"May 3, 2023 ,",By Karen Lema\nMay 3 (Reuters) – The United St...
5,‘TRUE FRIENDSHIP’ | Marcos cites need to ‘cont...,By Beatrice Puente,"May 3, 2023 ,","By Beatrice Puente\n(May 3, 2023) – President ..."
6,"US-PHILIPPINE TIES | Amid China pressure, US a...","By Jeff Mason, Trevor Hunnicutt and David Brun...","May 2, 2023 ,","By Jeff Mason, Trevor Hunnicutt and David Brun..."
7,"DIRECT LINE | Philippines, China to discuss fi...",(Reporting by Enrico Dela Cruz; Editing by Ale...,"May 2, 2023 ,","MANILA, May 1 (Reuters) – Philippines Presiden..."
8,DEFENSE POSTURE | Marcos’ US visit strategic t...,By Beatrice Puente,"May 2, 2023 ,","By Beatrice Puente\n(May 2, 2023) – President ..."
9,"IRONCLAD TIES | US, Philippines recommit to se...",(Reporting by Trevor Hunnicutt and David Brunn...,"May 2, 2023 ,","By Jeff Mason, Trevor Hunnicutt and David Brun..."


## Manila Standard


In [ ]:
# https://manilastandard.net/category/news/national

site = "Manila Standard"
data=pd.DataFrame(columns=['Statement','Link','Date'])
url = 'https://manilastandard.net/category/news/national'
print(url)

driver = startChromeDriver()
driver.get(url)
driver.implicitly_wait(30)

source = driver.page_source

time.sleep(3)
soup = BeautifulSoup(source, 'html.parser')

links = soup.find('div',{'class':'jeg_hero_wrapper'}).find_all('div',attrs={'class':'jeg_postblock_content'})

try:
    js_code = "arguments[0].scrollIntoView();"
    # The WebElement you want to scroll to
    element = WebDriverWait(driver, timeout=5).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[2]/div[5]/div/div[1]/div[2]/div/div[4]/div[1]/div/div[1]/div/h1')))
    # Execute the JS script
    driver.execute_script(js_code, element)
    time.sleep(2)  
except Exception as e:                                   
    error_type, error_obj, error_info = sys.exc_info()      
    print ('ERROR FOR LINK:',url)                          
    print (error_type, 'Line:', error_info.tb_lineno) 

linksPOST = soup.find('div', {'class':'jeg_main_content jeg_column col-sm-8'}).find_all('div',attrs={'class':'jeg_postblock_content'})

print(len(links)+len(linksPOST))

filename= f"{dir}{site}_{date.today()}_NEWS_LinkList.csv"    

for j in links:
    Statement = j.find('h2', {'class' : 'jeg_post_title'}).find('a').text.strip()
    Link = j.find('h2', {'class' : 'jeg_post_title'}).find('a')['href'].strip()
    Date = j.find('div', {'class': 'jeg_meta_date'}).text.strip()
    df_new_row = { 'Statement': Statement, 'Link': Link, 'Date': Date}
    data = data.append(pd.Series(df_new_row, index=data.columns[:len(df_new_row)]), ignore_index=True)

for j in linksPOST:
    Statement = j.find('h3', {'class' : 'jeg_post_title'}).find('a').text.strip()
    Link = j.find('h3', {'class' : 'jeg_post_title'}).find('a')['href'].strip()
    Date = j.find('div', {'class': 'jeg_meta_date'}).text.strip()
    df_new_row = { 'Statement': Statement, 'Link': Link, 'Date': Date}
    data = data.append(pd.Series(df_new_row, index=data.columns[:len(df_new_row)]), ignore_index=True)

driver.quit()
data.head()

https://manilastandard.net/category/news/national
15


,Statement,Link,Date
0,US firms keen on hiring Pinoy workers from Sudan,https://manilastandard.net/news/national/31432...,"May 5, 2023, 12:30 am"
1,DICT to launch SIM Check to fight scammers,https://manilastandard.net/news/national/31432...,"May 5, 2023, 12:25 am"
2,PNP to implement cigarette and vape ban among ...,https://manilastandard.net/news/national/31432...,"May 5, 2023, 12:20 am"
3,More people think ROTC should be optional than...,https://manilastandard.net/news/national/31432...,"May 4, 2023, 11:30 pm"
4,PNP prioritizes cases of missing ‘sabungeros ‘,https://manilastandard.net/news/314327726/pnp-...,"May 4, 2023, 11:20 pm"


In [ ]:
# https://manilastandard.net/category/news/top-stories

site = "Manila Standard"
url = 'https://manilastandard.net/category/news/top-stories'
print(url)

driver = startChromeDriver()
driver.get(url)
driver.implicitly_wait(10)

source = driver.page_source
time.sleep(3)
soup = BeautifulSoup(source, 'html.parser')

links = soup.find('div',{'class':'jeg_hero_wrapper'}).find_all('div',attrs={'class':'jeg_postblock_content'})

try:
    js_code = "arguments[0].scrollIntoView();"
    # The WebElement you want to scroll to
    element = WebDriverWait(driver, timeout=5).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div/div[1]/div[2]/div/div[4]/div[1]/div/div[1]/div/h1')))
    # Execute the JS script
    driver.execute_script(js_code, element)
    time.sleep(2)  
except Exception as e:                                   
    error_type, error_obj, error_info = sys.exc_info()      
    print ('ERROR FOR LINK:',url)                          
    print (error_type, 'Line:', error_info.tb_lineno) 

linksPOST = soup.find('div', {'class':'jeg_main_content jeg_column col-sm-8'}).find_all('div',attrs={'class':'jeg_postblock_content'})

print(len(links)+len(linksPOST))

filename= f"{dir}{site}_{date.today()}_NEWS_LinkList.csv"    

for j in links:
    Statement = j.find('h2', {'class' : 'jeg_post_title'}).find('a').text.strip()
    Link = j.find('h2', {'class' : 'jeg_post_title'}).find('a')['href'].strip()
    Date = j.find('div', {'class': 'jeg_meta_date'}).text.strip()
    df_new_row = { 'Statement': Statement, 'Link': Link, 'Date': Date}
    data = data.append(pd.Series(df_new_row, index=data.columns[:len(df_new_row)]), ignore_index=True)

for j in linksPOST:
    Statement = j.find('h3', {'class' : 'jeg_post_title'}).find('a').text.strip()
    Link = j.find('h3', {'class' : 'jeg_post_title'}).find('a')['href'].strip()
    Date = j.find('div', {'class': 'jeg_meta_date'}).text.strip()
    df_new_row = { 'Statement': Statement, 'Link': Link, 'Date': Date}
    data = data.append(pd.Series(df_new_row, index=data.columns[:len(df_new_row)]), ignore_index=True)


driver.quit()
data.head()

https://manilastandard.net/category/news/top-stories
15


,Statement,Link,Date
0,US firms keen on hiring Pinoy workers from Sudan,https://manilastandard.net/news/national/31432...,"May 5, 2023, 12:30 am"
1,DICT to launch SIM Check to fight scammers,https://manilastandard.net/news/national/31432...,"May 5, 2023, 12:25 am"
2,PNP to implement cigarette and vape ban among ...,https://manilastandard.net/news/national/31432...,"May 5, 2023, 12:20 am"
3,More people think ROTC should be optional than...,https://manilastandard.net/news/national/31432...,"May 4, 2023, 11:30 pm"
4,PNP prioritizes cases of missing ‘sabungeros ‘,https://manilastandard.net/news/314327726/pnp-...,"May 4, 2023, 11:20 pm"


In [ ]:
filename= f"{dir}{site}_{date.today()}_NEWS_LinkList.csv"    
data.to_csv(filename)

In [ ]:
def manilaStandardArticleScraper(url):
  print(url)
  row=[]
  try: 
    driver = startChromeDriver()
    driver.get(url)
    WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, 'jeg_post_title'))) 
    source = driver.page_source
    soup=BeautifulSoup(source, 'html.parser')

    
    Title = soup.find('h1',{'class':'jeg_post_title'}).text.strip()
    try:
        Author = soup.find('div', attrs={'class': 'jeg_meta_author coauthor'}).text.strip()
    except Exception as e:                                   
      error_type, error_obj, error_info = sys.exc_info()      
      print ('ERROR FOR LINK:',url)                          
      print (error_type, 'Line:', error_info.tb_lineno)
      Author = ""
        
    try:
        Date = soup.find('div', {'class' :'jeg_meta_date'}).text[:-10].strip() 
    except Exception as e:                                   
      error_type, error_obj, error_info = sys.exc_info()      
      print ('ERROR FOR LINK:',url)                          
      print (error_type, 'Line:', error_info.tb_lineno)
      Date = ""
    time.sleep(3)
    textList =soup.find('div',{'class':'content-inner'}).find_all("p")
    body = ""
    for t in textList:
        body += (t.text) +"\n" 
    row.extend((Title, Author, Date, body))
    
  except Exception as e:                                   
      error_type, error_obj, error_info = sys.exc_info()      
      print ('ERROR FOR LINK:',url)                          
      print (error_type, 'Line:', error_info.tb_lineno)
  driver.quit()
  return row

site = "Manila Standard"
data2 = pd.DataFrame(columns=["Title", "Author", "Date", "Text"])
filename= f"{dir+site}_{date.today()}NEWS.csv"

for index, r in data.iterrows():
  url = r["Link"] 
  row = manilaStandardArticleScraper(url)
  data2 = data2.append(pd.Series(row, index=data2.columns[:len(row)]), ignore_index=True)
  
data2.to_csv(filename)
data2

https://manilastandard.net/news/national/314327780/us-firms-keen-on-hiring-pinoy-workers-from-sudan.html
https://manilastandard.net/news/national/314327760/dict-to-launch-sim-check-to-fight-scammers.html
https://manilastandard.net/news/national/314327777/pnp-to-implement-cigarette-and-vape-ban-among-minors.html
https://manilastandard.net/news/national/314327728/more-people-think-rotc-should-be-optional-than-compulsory-sws.html
https://manilastandard.net/news/314327726/pnp-prioritizes-cases-of-missing-sabungeros.html
https://manilastandard.net/news/314327724/kadiwa-stores-post-over-p6-4m-sales-nationwide.html
https://manilastandard.net/news/national/314327719/exigency-plans-readied-for-mindoro-fishermen.html
https://manilastandard.net/news/national/314327763/ao-creates-independence-day-panel.html
https://manilastandard.net/news/314327752/comelec-disqualifies-legazpi-mayor-carmen-rosal.html
https://manilastandard.net/news/national/314327750/tolentino-wants-probe-on-govt-plans-vs-avian-fl

,Title,Author,Date,Text
0,US firms keen on hiring Pinoy workers from Sudan,byManila Standard,"May 5, 2023,",American companies have expressed interest in ...
1,DICT to launch SIM Check to fight scammers,byOrlan L. Mauricio,"May 5, 2023,",The Department of Information and Communicatio...
2,PNP to implement cigarette and vape ban among ...,byCharles Dantes,"May 5, 2023,",The Philippine National Police on Thursday ann...
3,More people think ROTC should be optional than...,byVito Barcelo,"May 4, 2023,",More Filipinos think that the Reserve Officers...
4,PNP prioritizes cases of missing ‘sabungeros ‘,byCharles Dantes,"May 4, 2023,",The Philippine National Police (PNP) has given...
5,Kadiwa stores post over P6.4m sales nationwide,byManila Standard,"May 4, 2023,",The government-run Kadiwa stores have generate...
6,Exigency plans readied for Mindoro fishermen,byJoel E. Zurbano,"May 4, 2023,",The Department of the Interior and Local Gover...
7,AO creates Independence Day panel,byVince Lopez,"May 4, 2023,",President Ferdinand Marcos Jr. has signed an a...
8,Comelec disqualifies Legazpi Mayor Carmen Rosal,byVito Barcelo,"May 4, 2023,",The Commission on Elections (Comelec) en banc ...
9,Tolentino wants probe on gov’t plans vs. avian...,byMacon Ramos-Araneta,"May 4, 2023,",Sen. Francis Tolentino wants a Senate probe on...


# MAIN Application

## GMA 

### site scraper

In [ ]:
# https://www.gmanetwork.com/news/archives/news-nation
#  https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python
site = "GMA"
gmadata=pd.DataFrame(columns=['Statement','Link','Date'])
url = 'https://www.gmanetwork.com/news/archives/news-nation/'
print(url)

driver = startChromeDriver()
driver.get(url)
driver.implicitly_wait(10)
source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')

dates = soup.find_all('li',{'class':'month_data'})

try:
    cnt = 220
    for panel in range(2):
        if (cnt<0):
            break
        for li in range(5):
            # //*[@id="ui-accordion-accordion-panel-0"]/ul/li[1]
            xPath = '//*[@id="ui-accordion-accordion-panel-'+str(panel)+'"]/ul/li['+str(li+1)+']'
            element = WebDriverWait(driver, timeout=30).until(EC.element_to_be_clickable((By.XPATH, xPath)))
            # driver.execute_script(js_code, element)
            element.click()
            print(xPath,'clicked')
            if (cnt<0):
                break
            try:
                SCROLL_PAUSE_TIME = 0.5
                # Get scroll height
                last_height = driver.execute_script("return document.body.scrollHeight")
                while cnt >0:
                    # Scroll down to bottom
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    # Wait to load page
                    time.sleep(SCROLL_PAUSE_TIME)
                    # Calculate new scroll height and compare with last scroll height
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    if new_height == last_height:
                        break
                    last_height = new_height
                    print('scrolling', cnt)
                    time.sleep(1)
                    cnt -= 1

                time.sleep(1)
                linksL=soup.find("ul", {'id' : 'grid_thumbnail_stories'}).find_all('li', {'class':'story left-grid'})
                linksR=soup.find("ul", {'id' : 'grid_thumbnail_stories'}).find_all('li', {'class':'story right-grid'})
                links = linksL + linksR
                print(xPath, len(links))

                for j in links:
                    Statement = j.find('div').find('div', {'class': 'story'}).find('div', {'class': 'story_title_holder'}).find('div', {'class' : 'story_title'}).text[:-21].strip()
                    Link = j.find('a')['href'].strip()
                    Date = j.find('div', {'class' : 'story_title_holder'}).find('div', {'class' : 'story_title'}).find('div', {'class': 'archive_date_time'}).text[:-8].strip()
                    df_new_row = { 'Statement': Statement, 'Link': Link, 'Date': Date}
                    gmadata = gmadata.append(pd.Series(df_new_row, index=gmadata.columns[:len(df_new_row)]), ignore_index=True)

            except Exception as e:                                   
                error_type, error_obj, error_info = sys.exc_info()      
                print ('ERROR FOR LINK:',url)                          
                print (error_type, 'Line:', error_info.tb_lineno)

 
except Exception as e:                                   
  error_type, error_obj, error_info = sys.exc_info()      
  print ('ERROR FOR LINK:',url)                          
  print (error_type, 'Line:', error_info.tb_lineno) 




filename= f"{dir}{site}_{date.today()}_NEWS_LinkList.csv"    
driver.quit()
gmadata.drop_duplicates(inplace=True)
gmadata.to_csv(filename)
gmadata.head()

https://www.gmanetwork.com/news/archives/news-nation/
//*[@id="ui-accordion-accordion-panel-0"]/ul/li[1] clicked
scrolling 220
scrolling 219
scrolling 218
scrolling 217
scrolling 216
scrolling 215
scrolling 214
scrolling 213
scrolling 212
scrolling 211
scrolling 210
scrolling 209
scrolling 208
scrolling 207
scrolling 206
scrolling 205
scrolling 204
scrolling 203
scrolling 202
scrolling 201
scrolling 200
scrolling 199
scrolling 198
scrolling 197
scrolling 196
scrolling 195
scrolling 194
scrolling 193
scrolling 192
scrolling 191
scrolling 190
scrolling 189
scrolling 188
scrolling 187
scrolling 186
scrolling 185
scrolling 184
scrolling 183
scrolling 182
scrolling 181
scrolling 180
scrolling 179
scrolling 178
scrolling 177
scrolling 176
scrolling 175
scrolling 174
scrolling 173
scrolling 172
scrolling 171
scrolling 170
scrolling 169
scrolling 168
scrolling 167
scrolling 166
scrolling 165
scrolling 164
scrolling 163
scrolling 162
scrolling 161
scrolling 160
scrolling 159
scrolling 158
scrol

KeyboardInterrupt: ignored

In [ ]:
filename= f"{dir}{site}_{date.today()}_NEWS_LinkList.csv"    
driver.quit()
gmadata.drop_duplicates(inplace=True)
gmadata.to_csv(filename)
gmadata.shape

(0, 3)

### artricle scraper

In [ ]:
def gmaArticleScraper(url):
    try: 
        driver = startChromeDriver()
        driver.get(url)
        WebDriverWait(driver, timeout=30).until(EC.visibility_of_element_located((By.CLASS_NAME, 'upper_article'))) 
        source = driver.page_source
        soup=BeautifulSoup(source, 'html.parser')

        row=[]
        try: 
            Title = soup.find('h1',{'class':'story_links'}).text.strip()
            Author = soup.find('div',attrs={'class':'main-byline'}).find('div').find('div').text.strip()
            Date = soup.find('div', attrs={'class': 'article-time'}).find('time')['datetime'][:10].strip() 
            textList =soup.find("div", {'class' :'story_main'}).find_all("p")
            body = ""
            for t in textList:
                if (t.find('p', class_="ad")):
                    pass
                else:
                    body += (t.text) +"\n" 
            row.extend((Title, Author, Date, body))
        except AttributeError:
            pass

    except Exception as e:                                   
        error_type, error_obj, error_info = sys.exc_info()      
        print ('ERROR FOR LINK:',url)                          
        print (error_type, 'Line:', error_info.tb_lineno)
    driver.quit()
    return row

site = "GMA"
gmadata2 = pd.DataFrame(columns=["Title", "Author", "Date", "Text"])
filename= f"{dir+site}_{date.today()}NEWS.csv"

df = pd.read_csv(f'{dir}GMA_2023-05-05_NEWS_LinkList.csv')
for index, r in df.iterrows():
  url = r["Link"] 
  print(index, url)
  row = gmaArticleScraper(url)
  gmadata2 = gmadata2.append(pd.Series(row, index=gmadata2.columns[:len(row)]), ignore_index=True)
  
gmadata2.to_csv(filename)
gmadata2.head()

,Title,Author,Date,Text
0,"Over 4,000 motorcycle road crashes recorded fr...","By JOVILAND RITA, GMA Integrated News",2023-05-05,Philippine National Police - Highway Patrol G...
1,CAAP shortens May 17 air traffic management sy...,"By TED CORDERO, GMA Integrated News",2023-05-05,The Civil Aviation Authority of the Philippin...
2,75 PDLs with mild symptoms remain in isolation...,By GMA Integrated News,2023-05-05,A total of 75 persons deprived of liberty (PD...
3,Marcos says disengagement with China 'not an o...,"By ANNA FELICIA BAJO, GMA Integrated News",2023-05-05,"WASHINGTON — President Ferdinand ""Bongbong"" M..."
4,Marcos to meet with British PM on the sideline...,"By ANNA FELICIA BAJO, GMA Integrated News",2023-05-05,"WASHINGTON — President Ferdinand ""Bongbong"" M..."


## Rappler

### site scraper

In [ ]:
# https://www.rappler.com/nation/national-news/
site = "Rappler"
rapplerData=pd.DataFrame(columns=['Statement','Link','Date'])

pagesToGet= 550
driver = startChromeDriver()

for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://www.rappler.com/nation/national-news/page/'+str(page)+'/'
    print(url)
    driver.get(url)
    driver.implicitly_wait(10)

    source = driver.page_source
    WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, 'post-card__more')))
    soup = BeautifulSoup(source, 'html.parser')
    links = soup.find_all('div',attrs={'class':'post-card__more-secondary-story'})
    print(len(links))

    filename= f"{dir}{site}_{date.today()}_NEWS_LinkList.csv"    

    for j in links:
        Statement = j.find('h3', {'class' : 'post-card__title'}).find('a').text.strip()
        Link = j.find("h3", {'class' : 'post-card__title'}).find('a')['href'].strip()
        Date = j.find('time')['datetime'][:10].strip()
        df_new_row = { 'Statement': Statement, 'Link': Link, 'Date': Date}
        rapplerData = rapplerData.append(pd.Series(df_new_row, index=rapplerData.columns[:len(df_new_row)]), ignore_index=True)

driver.quit()
rapplerData.drop_duplicates(inplace=True)
# rapplerData.to_csv(filename)
rapplerData.head()

processing page : 1
https://www.rappler.com/nation/national-news/page/1/
5
processing page : 2
https://www.rappler.com/nation/national-news/page/2/
4
processing page : 3
https://www.rappler.com/nation/national-news/page/3/
4
processing page : 4
https://www.rappler.com/nation/national-news/page/4/
4
processing page : 5
https://www.rappler.com/nation/national-news/page/5/
4
processing page : 6
https://www.rappler.com/nation/national-news/page/6/
4
processing page : 7
https://www.rappler.com/nation/national-news/page/7/
4
processing page : 8
https://www.rappler.com/nation/national-news/page/8/
4
processing page : 9
https://www.rappler.com/nation/national-news/page/9/
4
processing page : 10
https://www.rappler.com/nation/national-news/page/10/
4
processing page : 11
https://www.rappler.com/nation/national-news/page/11/
4
processing page : 12
https://www.rappler.com/nation/national-news/page/12/
4
processing page : 13
https://www.rappler.com/nation/national-news/page/13/
4
processing page :

,Statement,Link,Date
0,Marcos the party-going schoolboy returns to UK...,https://www.rappler.com/world/global-affairs/m...,2023-05-05
1,"Ablan confirms resignation from DepEd, denies ...",https://www.rappler.com/nation/kris-ablan-conf...,2023-05-05
2,WATCH: Paano naibenta sa labas ang DepEd lapto...,https://www.rappler.com/newsbreak/podcasts-vid...,2023-05-05
3,"In unprecedented move, Filipino journalist har...",https://www.rappler.com/nation/journalist-cong...,2023-05-05
4,Marcos says Philippines bases could be ‘useful...,https://www.rappler.com/nation/marcos-jr-phili...,2023-05-05


In [ ]:
rapplerData.shape
rapplerData.to_csv(filename)

### artricle scraper

In [ ]:
def rapplerArticleScraper(url):
#   print(url)
  try: 
    driver = startChromeDriver()
    driver.get(url)
    WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, 'post-single__header'))) 
    source = driver.page_source
    soup=BeautifulSoup(source, 'html.parser')

    row=[]
    Title = soup.find('h1',{'class':'post-single__title'}).text.strip()
    try:
        Author = soup.find('div', attrs={'class': 'post-single__authors'}).text.strip()
    except AttributeError:
        Author = ""
        
    try:
        Date = soup.find('time')['datetime'][:10].strip() 
    except Exception as e:                                   
      error_type, error_obj, error_info = sys.exc_info()      
      print ('ERROR FOR LINK:',url)                          
      print (error_type, 'Line:', error_info.tb_lineno)
      Date = ""

    textList =soup.find("div", {'class' :'post-single__content entry-content'}).find_all("p")
    body = ""
    for t in textList:
        body += (t.text) +"\n" 
    row.extend((Title, Author, Date, body))
    
  except Exception as e:                                   
      error_type, error_obj, error_info = sys.exc_info()      
      print ('ERROR FOR LINK:',url)                          
      print (error_type, 'Line:', error_info.tb_lineno)
  driver.quit()
  return row

site = "Rappler"
rapplerData2 = pd.DataFrame(columns=["Title", "Author", "Date", "Text"])
filename= f"{dir+site}_{date.today()}NEWS.csv"

df = pd.read_csv(f'{dir}Rappler_2023-05-05_NEWS_LinkList.csv')
for index, r in df.iterrows():
  url = r["Link"] 
  print(index, url)
  row = rapplerArticleScraper(url)
  rapplerData2 = rapplerData2.append(pd.Series(row, index=rapplerData2.columns[:len(row)]), ignore_index=True)
  
rapplerData2.to_csv(filename)
rapplerData2

https://www.rappler.com/nation/comelec-wish-list-2025-automated-elections/
https://www.rappler.com/nation/united-states-issues-guidelines-defending-philippines-south-china-sea-attack/
https://www.rappler.com/nation/follow-audience-social-media-platforms-tiktok-journalists-urge-colleagues/
https://www.rappler.com/nation/covid-19-cases-rise-bucor-suspends-visitation-new-bilibid-prison/
https://www.rappler.com/nation/updates-ferdinand-marcos-jr-official-visit-united-states-2023/
https://www.rappler.com/nation/joey-salceda-flags-government-agencies-slow-disbursement-funds-2023/
https://www.rappler.com/nation/video-challenges-pursuit-press-freedom-philippines/
https://www.rappler.com/nation/sandiganbayan-decision-graft-case-vs-ex-qc-mayor-herbert-bautista/
https://www.rappler.com/nation/marcos-jr-biden-meet-flexes-optimism-philippines-united-states-ties/
https://www.rappler.com/nation/amid-china-pressure-united-states-philippines-recommit-security-alliance/
https://www.rappler.com/nation/vi

,Title,Author,Date,Text
0,"Elections 2025: Stamping pens, new machine typ...",Dwight de Leon,2023-05-04,"MANILA, Philippines – The Commission on Electi..."
1,US issues guidelines on defending Philippines ...,Reuters,2023-05-04,"MANILA, Philippines – The United States has la..."
2,"Follow audience on platforms like TikTok, jour...",James Patrick Cruz,2023-05-04,"MANILA, Philippines – Media should follow thei..."
3,"As COVID-19 cases rise, BuCor suspends visitat...",Jairo Bolledo,2023-05-04,"MANILA, Philippines – In light of the rising C..."
4,LIVE UPDATES: President Marcos’ official visit...,Bea Cupin,2023-05-03,Philippine President Ferdinand Marcos Jr. head...
5,House leader flags government agencies’ slow f...,Dwight de Leon,2023-05-03,"MANILA, Philippines – House ways and means pan..."
6,WATCH: Rapplers share their challenges in purs...,Vixey Lema,2023-05-03,"MANILA, Philippines – What does it take to be ..."
7,Sandiganbayan affirms graft case vs ex-QC mayo...,Rappler.com,2023-05-02,"MANILA, Philippines – The Sandiganbayan, the c..."
8,‘Welcome back’: Marcos-Biden meeting flexes ‘o...,Bea Cupin,2023-05-02,"WASHINGTON DC, USA – When US President Joe Bid..."
9,"Amid China pressure, US, Philippines recommit ...",Reuters,2023-05-02,"WASHINGTON DC, USA – The US and the Philippine..."


## Sunstar Philippines

### site scraper

In [ ]:
# https://www.sunstar.com.ph/morearticles/manila/local-news?pgno=1

site = "SunStar"
starData=pd.DataFrame(columns=['Statement','Link','Date'])

pagesToGet= 200
for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://www.sunstar.com.ph/morearticles/manila/local-news?pgno='+str(page)
    print(url)

    driver = startChromeDriver()
    driver.get(url)
    # showscreen(driver)
    # WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'col-md-12 noPadding col-xs-12')))
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    links = soup.find_all('div',attrs={'class':'article-info'})
    print(len(links))

    filename= f"{dir}{site}_{date.today()}_NEWS_LinkList.csv"    

    for j in links:
        try:
            Statement = j.find('h2', {'class' : 'article-title'}).find('a').text.strip()
            Link = j.find('h2', {'class' : 'article-title'}).find('a')['href'].strip()
            Date = j.find('div', {'class': 'article-date'}).text.strip()
            df_new_row = { 'Statement': Statement, 'Link': Link, 'Date': Date}
            starData = starData.append(pd.Series(df_new_row, index=starData.columns[:len(df_new_row)]), ignore_index=True)
        except Exception as e:                                   
            error_type, error_obj, error_info = sys.exc_info()      
            print ('ERROR FOR LINK:',url)                          
            print (error_type, 'Line:', error_info.tb_lineno)

driver.quit()
print(starData.shape)
starData.drop_duplicates(inplace=True)
starData.to_csv(filename)
starData.head()

processing page : 1
https://www.sunstar.com.ph/morearticles/manila/local-news?pgno=1
20
processing page : 2
https://www.sunstar.com.ph/morearticles/manila/local-news?pgno=2
20
processing page : 3
https://www.sunstar.com.ph/morearticles/manila/local-news?pgno=3
20
processing page : 4
https://www.sunstar.com.ph/morearticles/manila/local-news?pgno=4
20
processing page : 5
https://www.sunstar.com.ph/morearticles/manila/local-news?pgno=5
20
processing page : 6
https://www.sunstar.com.ph/morearticles/manila/local-news?pgno=6
20
processing page : 7
https://www.sunstar.com.ph/morearticles/manila/local-news?pgno=7
20
processing page : 8
https://www.sunstar.com.ph/morearticles/manila/local-news?pgno=8
20
processing page : 9
https://www.sunstar.com.ph/morearticles/manila/local-news?pgno=9
20
processing page : 10
https://www.sunstar.com.ph/morearticles/manila/local-news?pgno=10
20
processing page : 11
https://www.sunstar.com.ph/morearticles/manila/local-news?pgno=11
20
processing page : 12
https:/

,Statement,Link,Date
0,Marcos accepts courtesy resignation of 2 senio...,https://www.sunstar.com.ph/article/1960506/man...,"May 05, 2023"
1,"DOH reiterates call for kids to get measles, p...",https://www.sunstar.com.ph/article/1960483/man...,"May 05, 2023"
2,Octa admits positivity rate does not show whol...,https://www.sunstar.com.ph/article/1960482/man...,"May 05, 2023"
3,DMW: Japanese firms ready to employ Sudan OFWs,https://www.sunstar.com.ph/article/1960475/man...,"May 05, 2023"
4,146 OFWs still in Sudan,https://www.sunstar.com.ph/article/1960474/man...,"May 05, 2023"


In [ ]:
starData.shape
# 600 no duplicates 

(600, 3)

### artricle scraper

In [27]:
def sunStarArticleScraper(url):
  print(url)
  row=[]
  try: 
    driver = startChromeDriver()
    # driver.get(url)
    # WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.CLASS_NAME, 'article-body'))) 
    time.sleep(2)
    # source = driver.page_source
    soup=BeautifulSoup(extract_source(url).text, 'html.parser')
    
    Title = soup.find('h1',{'class':'title-article'}).text.strip()
    try:
        Author = soup.find('div', attrs={'class': 'article-byline'}).text.strip()
    except AttributeError:
        Author = ""
        
    try:
        Date = soup.find('div', {'class': 'article-date'}).text.strip() 
    except Exception as e:                                   
      error_type, error_obj, error_info = sys.exc_info()      
      print ('ERROR FOR LINK:',url)                          
      print (error_type, 'Line:', error_info.tb_lineno)
      Date = ""

    body = soup.find('div', {'class': 'article-body'}).text.strip()
    row.extend((Title, Author, Date, body))
    
  except Exception as e:                                   
      error_type, error_obj, error_info = sys.exc_info()      
      print ('ERROR FOR LINK:',url)                          
      print (error_type, 'Line:', error_info.tb_lineno)

#   driver.quit()
  return row

site = "SunStar"
starData2 = pd.DataFrame(columns=["Title", "Author", "Date", "Text"])
filename= f"{dir+site}_{date.today()}NEWS.csv"

df = pd.read_csv(f'{dir}SunStar_2023-05-05_NEWS_LinkList.csv')
for index, r in df.iterrows():
  url = r["Link"] 
  row = sunStarArticleScraper(url)
  starData2 = starData2.append(pd.Series(row, index=starData2.columns[:len(row)]), ignore_index=True)
  
starData2.to_csv(filename)
starData2.head()

https://www.sunstar.com.ph/article/1960506/manila/local-news/marcos-accepts-courtesy-resignation-of-2-senior-police-officers


[WDM] - Downloading: 100%|██████████| 5.53M/5.53M [00:00<00:00, 5.95MB/s]


https://www.sunstar.com.ph/article/1960483/manila/local-news/doh-reiterates-call-for-kids-to-get-measles-polio-jabs
https://www.sunstar.com.ph/article/1960482/manila/local-news/octa-admits-positivity-rate-does-not-show-whole-covid-picture
https://www.sunstar.com.ph/article/1960475/manila/local-news/dmw-japanese-firms-ready-to-employ-sudan-ofws
https://www.sunstar.com.ph/article/1960474/manila/local-news/146-ofws-still-in-sudan
https://www.sunstar.com.ph/article/1960465/manila/local-news/ph-to-continue-discussions-about-malampaya-with-china
https://www.sunstar.com.ph/article/1960460/manila/local-news/comelec-affirms-disqualification-of-legazpi-city-mayor
https://www.sunstar.com.ph/article/1960459/manila/local-news/1090-foreigners-filipinos-rescued-from-alleged-human-trafficking-operation
https://www.sunstar.com.ph/article/1960458/manila/local-news/papal-nuncio-urges-ph-church-to-put-premium-on-missionary-outreach-efforts
https://www.sunstar.com.ph/article/1960456/manila/local-news/corre

KeyboardInterrupt: ignored

## Philippine News Agency

### site scraper

In [ ]:
# https://www.pna.gov.ph/categories/national

site = "PhilippineNewsAgency"
pnaData=pd.DataFrame(columns=['Statement','Link','Date'])

pagesToGet= 110
for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://www.pna.gov.ph/categories/national?p='+str(page)
    print(url)
    driver = startChromeDriver()
    driver.get(url)
    driver.implicitly_wait(10)
    WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, 'articles')))

    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    links = soup.find_all('div',attrs={'class':'article media'})
    print(len(links))

    filename= f"{dir}{site}_{date.today()}_NEWS_LinkList.csv"    

    for j in links:
        try:
            Statement = j.find('h3', {'class' : 'media-heading'}).find('a').text.strip()
            Link = 'https://www.pna.gov.ph'+j.find('h3', {'class' : 'media-heading'}).find('a')['href'].strip()
            Date = j.find('span', {'class': 'date'}).text.strip()
            df_new_row = { 'Statement': Statement, 'Link': Link, 'Date': Date}
            pnaData = pnaData.append(pd.Series(df_new_row, index=pnaData.columns[:len(df_new_row)]), ignore_index=True)
        except Exception as e:                                   
            error_type, error_obj, error_info = sys.exc_info()      
            print ('ERROR FOR LINK:',url)                          
            print (error_type, 'Line:', error_info.tb_lineno)

driver.quit()
pnaData.drop_duplicates(inplace=True)
# pnaData.to_csv(filename)
pnaData.head()

processing page : 1
https://www.pna.gov.ph/categories/national?p=1


[WDM] - Downloading: 100%|██████████| 5.53M/5.53M [00:00<00:00, 36.9MB/s]


20
processing page : 2
https://www.pna.gov.ph/categories/national?p=2
20
processing page : 3
https://www.pna.gov.ph/categories/national?p=3
20
processing page : 4
https://www.pna.gov.ph/categories/national?p=4
20
processing page : 5
https://www.pna.gov.ph/categories/national?p=5
20
processing page : 6
https://www.pna.gov.ph/categories/national?p=6
20
processing page : 7
https://www.pna.gov.ph/categories/national?p=7
20
processing page : 8
https://www.pna.gov.ph/categories/national?p=8
20
processing page : 9
https://www.pna.gov.ph/categories/national?p=9
20
processing page : 10
https://www.pna.gov.ph/categories/national?p=10
20
processing page : 11
https://www.pna.gov.ph/categories/national?p=11
20
processing page : 12
https://www.pna.gov.ph/categories/national?p=12
20
processing page : 13
https://www.pna.gov.ph/categories/national?p=13
20
processing page : 14
https://www.pna.gov.ph/categories/national?p=14
20
processing page : 15
https://www.pna.gov.ph/categories/national?p=15
20
proce

,Statement,Link,Date
0,"MIAA, Meralco begins NAIA T3 electrical audit",https://www.pna.gov.ph/articles/1200883,"May 5, 2023, 10:02 pm"
1,"Pag-IBIG, Singaporean firm eye building ‘net-z...",https://www.pna.gov.ph/articles/1200882,"May 5, 2023, 8:03 pm"
2,DMW assures aid for OFWS from Sudan who wanted...,https://www.pna.gov.ph/articles/1200873,"May 5, 2023, 7:25 pm"
3,PNP chief fetes HPG's vital role in upholding ...,https://www.pna.gov.ph/articles/1200870,"May 5, 2023, 6:08 pm"
4,"LTO chief: 'Fixers,' fake docs jeopardize publ...",https://www.pna.gov.ph/articles/1200865,"May 5, 2023, 6:07 pm"


In [ ]:
pnaData.shape

(2200, 3)

### artricle scraper

In [25]:
def pnaArticleScraper(url):
  
  row=[]
  try: 
    # driver = startChromeDriver()
    # driver.get(url)
    # WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, 'page-content'))) 

    # source = driver.page_source
    soup=BeautifulSoup(extract_source(url).text, 'html.parser')
    time.sleep(1)
    Title = soup.find('h1').text.strip()

    try:
        Author = soup.find('span', attrs={'class': 'd-block author mb-2'}).text.strip()
    except AttributeError:
        Author =''
    except Exception as e:                                   
      error_type, error_obj, error_info = sys.exc_info()      
      print ('ERROR FOR LINK:',url)                          
      print (error_type, 'Line:', error_info.tb_lineno)
      Author = ""
        
    try:
        Date = soup.find('div', {'class': 'col-sm-6 cell-1'}).find('span', {'class': 'date published'}).text[:-8].strip()[:-1] 
    except Exception as e:                                   
      error_type, error_obj, error_info = sys.exc_info()      
      print ('ERROR FOR LINK:',url)                          
      print (error_type, 'Line:', error_info.tb_lineno)
      Date = ""
    # print(Date[:-1])
    textList = soup.find('div', {'class': 'page-content'}).find_all('p')
    body = ""
    for t in textList:
        body += (t.text) +"\n" 
    row.extend((Title, Author, Date, body))
    
  except Exception as e:                                   
      error_type, error_obj, error_info = sys.exc_info()      
      print ('ERROR FOR LINK:',url)                          
      print (error_type, 'Line:', error_info.tb_lineno)

#   driver.quit()
  return row

site = "PhilippineNewsAgency"
pnaData2 = pd.DataFrame(columns=["Title", "Author", "Date", "Text"])
filename= f"{dir+site}_{date.today()}NEWS.csv"

df = pd.read_csv(f'{dir}PhilippineNewsAgency_2023-05-05_NEWS_LinkList.csv')
for index, r in df.iterrows():
  url = r["Link"] 
  print(index, url)
  row = pnaArticleScraper(url)
  pnaData2 = pnaData2.append(pd.Series(row, index=pnaData2.columns[:len(row)]), ignore_index=True)

# pnaData2.to_csv(filename)
pnaData2.head()

0 https://www.pna.gov.ph/articles/1200883
1 https://www.pna.gov.ph/articles/1200882
2 https://www.pna.gov.ph/articles/1200873
3 https://www.pna.gov.ph/articles/1200870
4 https://www.pna.gov.ph/articles/1200865
5 https://www.pna.gov.ph/articles/1200869
6 https://www.pna.gov.ph/articles/1200857
7 https://www.pna.gov.ph/articles/1200855
8 https://www.pna.gov.ph/articles/1200853
9 https://www.pna.gov.ph/articles/1200842
10 https://www.pna.gov.ph/articles/1200837
11 https://www.pna.gov.ph/articles/1200830
12 https://www.pna.gov.ph/articles/1200848
13 https://www.pna.gov.ph/articles/1200836
14 https://www.pna.gov.ph/articles/1200822
15 https://www.pna.gov.ph/articles/1200833
16 https://www.pna.gov.ph/articles/1200829
17 https://www.pna.gov.ph/articles/1200806
18 https://www.pna.gov.ph/articles/1200803
19 https://www.pna.gov.ph/articles/1200802
20 https://www.pna.gov.ph/articles/1200800
21 https://www.pna.gov.ph/articles/1200796
22 https://www.pna.gov.ph/articles/1200797
23 https://www.pna.go

,Title,Author,Date,Text
0,"MIAA, Meralco begins NAIA T3 electrical audit",,"May 5, 2023",File photo\nMANILA – The Manila International ...
1,"Pag-IBIG, Singaporean firm eye building ‘net-z...",,"May 5, 2023","MANILA – Marilene Acosta, the CEO of Pag-IBIG ..."
2,DMW assures aid for OFWS from Sudan who wanted...,By Marita Moaje,"May 5, 2023",HOME. A total of 74 repatriates from war-torn ...
3,PNP chief fetes HPG's vital role in upholding ...,By Christopher Lloyd Caliwan,"May 5, 2023",PNP chief Gen. Benjamin Acorda Jr. (PNA photo ...
4,"LTO chief: 'Fixers,' fake docs jeopardize publ...",By Raymond Carl Dela Cruz,"May 5, 2023",MANILA – The chief of the Land Transportation ...


## Manila Bulletin

https://www.scrapingbee.com/webscraping-questions/selenium/how-to-scroll-to-element-selenium/#:~:text=You%20can%20scroll%20to%20an,the%20execute_script%20as%20an%20argument.

### site scraper

In [ ]:
## Manila bulletin
# News link getter for National News
# Simulate clicking the more button in MB site, and then extract all the news link

site = "manilaBulletin"
mbData = pd.DataFrame(columns=['Statement','Link','Date'])
url = 'https://mb.com.ph/category/national'
print(url)

driver = startChromeDriver()
driver.get(url)
driver.implicitly_wait(10)

moreClicks = 405
for x in range(moreClicks):
  try:
    js_code = "arguments[0].scrollIntoView();"
    # The WebElement you want to scroll to
    # element = driver.find_element(By.XPATH, '//*[@id="app"]/div/main/div/div/div/div[3]/div[2]/div[2]/div/div[2]/button')
    element = WebDriverWait(driver, timeout=5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/div/main/div/div/div/div[3]/div[2]/div[2]/div/div[2]/button')))
    # Execute the JS script
    driver.execute_script(js_code, element)
    element.click()
    print(x,'clicked')
    # showscreen(driver)
    # time.sleep(2)  
  except Exception as e:                                   
      error_type, error_obj, error_info = sys.exc_info()      
      print ('ERROR FOR LINK:',url)                          
      print (error_type, 'Line:', error_info.tb_lineno)  

source = driver.page_source

soup = BeautifulSoup(source, 'html.parser')
links=soup.find("div", {'class' : 'article-list mx-auto'}).find_all('div',attrs={'class':'row mb-5'})
print(len(links))

filename= f"{dir}{site}_{date.today()}_NEWS_LinkList.csv"    

for j in links:
    Statement = j.find('h4', {'class' : 'mb-font-article-title mt-0 mb-1'}).find('a').text.strip()
    Link = "https://mb.com.ph"+j.find("h4", {'class' : 'mb-font-article-title mt-0 mb-1'}).find('a')['href'].strip()
    Date = j.find('div', {'class': 'ml-2'}).text.strip()
    df_new_row = { 'Statement': Statement, 'Link': Link, 'Date': Date}
    mbData = mbData.append(pd.Series(df_new_row, index=mbData.columns[:len(df_new_row)]), ignore_index=True)

driver.quit()
mbData.drop_duplicates(inplace=True)
# mbData.to_csv(filename)
mbData.head()

NameError: ignored

In [26]:
mbData.shape

NameError: ignored

### article scraper

In [ ]:
# Article Reader from LINK
def manilabulletinArticleScraper(url):
    row=[]
    try:
        time.sleep(1)
        soup=BeautifulSoup(extract_source(url).text, 'html.parser')

        text=soup.find("div", {'class' : 'col-md-8 col-xl-8 col-12'})  
        Title = text.find('h1',{'class':'pt-3 mb-font-article-title'}).text.strip()
        Author = text.find('div',attrs={'class':'mb-font-author-name overflow-nowrap'}).find('a').text.strip()
        Date = text.find('div', attrs={'class': 'pt-0'}).text[:-8]
            
        body = ""
        textList =  soup.find('div', {'class':'pt-8 custom-article-body mb-font-article-body'}).find_all('p')

        for t in textList:
            body += (t.text.replace('\u200b', '')) +"\n" 
        row.extend((Title, Author, Date, body))

    except Exception as e:                                   
        error_type, error_obj, error_info = sys.exc_info()      
        print ('ERROR FOR LINK:',url)                          
        print (error_type, 'Line:', error_info.tb_lineno)
    return row

site = "manilaBulletin"
mbData2 = pd.DataFrame(columns=["Title", "Author", "Date", "Text"])
filename= f"{dir}{site}_{date.today()}_NEWS.csv"

df = pd.read_csv(f'{dir}manilaBulletin_2023-05-05_NEWS_LinkList.csv')
for index, r in df.iterrows():
    url = r["Link"] 
    print(index, url)
    row = manilabulletinArticleScraper(url)
    mbData2 = mbData2.append(pd.Series(row, index=mbData2.columns[:len(row)]), ignore_index=True)
  
# mbData2.to_csv(filename)
mbData2.head()

0 https://mb.com.ph/2023/5/5/thank-you-gov-cong-roman-mourns-passing-of-gov-carlos-padilla
1 https://mb.com.ph/2023/5/5/p5-m-reward-offered-to-find-missing-chinese-passenger-of-sunken-dive-yacht
2 https://mb.com.ph/2023/5/5/hontiveros-nuclear-energy-a-terrible-false-solution-to-ph-power-woes
3 https://mb.com.ph/2023/5/5/you-go-girl-pro-sports-solon-hails-kaila-napolis-for-winning-ph-s-1st-gold-in-2023-sea-games
4 https://mb.com.ph/2023/5/5/expedite-release-of-excellence-in-teacher-education-act-irr-agencies-told
5 https://mb.com.ph/2023/5/5/nwrb-keeps-an-eye-on-angat-dam-amid-el-nino-alert-2
6 https://mb.com.ph/2023/5/5/mb-daily-news-summary-42
7 https://mb.com.ph/2023/5/5/keeping-medical-frontliners-in-ph-health-workers-asks-gov-t-anew-for-salary-increase
8 https://mb.com.ph/2023/5/5/woman-arrested-for-extorting-p9-9-m-from-lawyer
9 https://mb.com.ph/2023/5/5/bfar-gears-up-for-el-nino-to-ensure-ample-food-supply
10 https://mb.com.ph/2023/5/5/ph-to-keep-negotiating-with-china-over-mala

,Title,Author,Date,Text
0,'Thank you gov': Cong Roman mourns passing of ...,Ellson Quismorio,"May 5, 2023",Bataan 1st district Rep. Geraldine Roman paid ...
1,P5-M reward offered to find missing Chinese pa...,Martin Sadongdong,"May 5, 2023",A P5-million reward was offered to anyone who ...
2,"Hontiveros: Nuclear energy a terrible, false s...",Hannah Torregoza,"May 5, 2023","\nSenator Risa Hontiveros on Friday, May 5 qu..."
3,You go girl! Pro-sports solon hails Kaila Napo...,Ellson Quismorio,"May 5, 2023",The first one is always special.\nPuwersa ng B...
4,Expedite release of Excellence in Teacher Educ...,Merlina Hernando-Malipot,"May 5, 2023",To further strengthen teacher education in the...


In [ ]:
print(filename)
mbData2
# mbData2.to_csv(filename)

/content/gdrive/MyDrive/THESIS-MS/Git-Thesis22-23/Official/manilaBulletin_2023-05-05_NEWS.csv
